In [1]:
##nced Object-Oriented Programming (OOP)
# - Metaclasses- Abstract Base Classes (abc module)
# - Multiple inheritance and MRO (Method Resolution Order)
# - Operator overloading (__add__, __str__, etc.)
# - Descriptors and properties

What is a Metaclass?
A metaclass is "the class of a class".
It defines how a class is created — like setting rules for all blueprints.

Every time you define a class like this:

In [ ]:
class Starship:
    pass

# Structure of a Metaclass

In [2]:
class MyMeta(type):
    def __new__(cls, name, bases, attrs):
        # You can modify the class before it's created
        print(f"Creating class {name}")
        return super().__new__(cls, name, bases, attrs)

In [3]:
class MyClass(metaclass=MyMeta):
    pass

Creating class MyClass


In [4]:
class StarshipMeta(type):
    def __new__(cls, name, bases, attrs):
        print(f"Blueprint ship {name}")
        if 'launch' not in attrs:
            raise TypeError("Starship must have a launch method")
        return type.__new__(cls, name, bases, attrs)

In [5]:
class Starship(metaclass=StarshipMeta):
    def launch(self):
        print("Launch sequence initiated")

Blueprint ship Starship


In [8]:
obj = Starship()
obj.launch()    

Launch sequence initiated


In [10]:
class Starship(metaclass=StarshipMeta):
    def Launch(self):
        print("Launch sequence initiated")


Blueprint ship Starship


TypeError: Starship must have a launch method

In [15]:
class RobotMeta(type):
    def __new__(cls, name, bases, attrs):
        if 'name' not in attrs:
            raise TypeError("Robots must have a name!")
        if 'status' not in attrs:
            attrs['status'] = "active"
        return super().__new__(cls, name, bases, attrs)

class Robot(metaclass=RobotMeta):
    def name(self):
        return self._name   

print(Robot.status)  

active


You are a junior engineer at the Galactic Starship Academy , where all cadets must design their own starships.

But there's a problem:
Some cadets are building ships that can't even launch or navigate properly!

To solve this, the academy introduces Basic Rules for Every Ship :

Must have a launch() method
Must have a navigate(destination) method
These rules are not tied to any specific type of ship — they apply to all ships , whether they're Fighters, Explorers, or Cargo Haulers.

So the academy creates a template class called Starship that defines these rules. But you can’t build a ship just from the template — it’s too generic.

In [16]:
from abc import ABC, abstractmethod

class Starship(ABC):
    @abstractmethod
    def launch(self):
        pass

    @abstractmethod
    def navigate(self, destination):
        pass

In [17]:
# ✅ This one follows the rules
class FighterShip(Starship):
    def launch(self):
        print("🛸 Fighter launching engines...")

    def navigate(self, destination):
        print(f"🎯 Navigating to {destination}")

In [18]:
# ❌ This one breaks the rules
class BrokenShip(Starship):
    def launch(self):
        print("❌ Incomplete ship trying to launch...")

In [19]:
falcon = FighterShip()
falcon.launch()
falcon.navigate("Mars")

🛸 Fighter launching engines...
🎯 Navigating to Mars


In [20]:
ghost = BrokenShip()
ghost.launch()

TypeError: Can't instantiate abstract class BrokenShip with abstract method navigate

In [21]:
from abc import ABC, abstractmethod

class SpaceVehicle(ABC):
    @abstractmethod
    def takeoff(self):
        pass

    @abstractmethod
    def land(self):
        pass

In [22]:
class Shuttle(SpaceVehicle):
    def takeoff(self):
        print("🚀 Shuttle taking off...")

    def land(self):
        print("🛫 Shuttle landing..."   )

🧭 What is MRO? (Method Resolution Order)
When a class inherits from multiple parents, Python uses a special rule to decide which parent’s method to run first .

This rule is called Method Resolution Order (MRO) .

Python uses the C3 Linearization Algorithm to determine this order — don’t worry about the math; just remember it gives us a clear path through the inheritance tree.

In [23]:
class SpaceEngine:
    def power_on(self):
        print("🔋 Engine powered on")

class WeaponsSystem:
    def power_on(self):
        print("🎯 Weapon system armed")

class Starship(SpaceEngine, WeaponsSystem):
    pass

In [24]:
s = Starship()
s.power_on()

🔋 Engine powered on


In [25]:
print(Starship.__mro__)

(<class '__main__.Starship'>, <class '__main__.SpaceEngine'>, <class '__main__.WeaponsSystem'>, <class 'object'>)


In [26]:
class Starship:
    def __init__(self, name, power):
        self.name = name
        self.power = power

1️⃣ __init__ – Constructor
Used to initialize new objects.

In [27]:
def __init__(self, name, power):
    self.name = name
    self.power = power

__str__ – Human-readable string

In [1]:
def __str__(self):
    return f"🚀 {self.name} | Power: {self.power} MW"

__repr__ – Unambiguous representation
Used for debugging, shown in consoles or logs.

In [2]:
def __repr__(self):
    return f"Starship(name='{self.name}', power={self.power})"

__add__ – Addition operator +

In [3]:
def __add__(self, other):
    return Starship(f"{self.name}+{other.name}", self.power + other.power)

__sub__ – Subtraction operator -

In [4]:
def __sub__(self, other):
    return Starship(f"{self.name}-{other.name}", max(0, self.power - other.power))

In [5]:
class Starship:
    def __init__(self, name, power):
        self.name = name
        self.power = power

    def __str__(self):
        return f"🚀 {self.name} | Power: {self.power} MW"

    def __repr__(self):
        return f"Starship(name='{self.name}', power={self.power})"

    def __add__(self, other):
        return Starship(f"{self.name}+{other.name}", self.power + other.power)

    def __sub__(self, other):
        return Starship(f"{self.name}-{other.name}", max(0, self.power - other.power))

    def __mul__(self, factor):
        return Starship(f"{self.name}*{factor}", self.power * factor)

    def __eq__(self, other):
        return self.power == other.power

    def __lt__(self, other):
        return self.power < other.power

    def __len__(self):
        return self.power * 2  # Meters

    def __call__(self):
        print(f"🛸 {self.name} launching manually...")


# Test it out!
falcon = Starship("Millennium Falcon", 750)
viper = Starship("Viper Mk VIII", 600)

# Use overloaded operators
print(falcon)           # str
print(repr(falcon))     # repr
print(falcon + viper)   # add
print(falcon - viper)   # sub
print(falcon * 2)       # mul
print(falcon == viper)  # eq
print(falcon > viper)   # lt (Python infers > from <)
print(len(falcon))      # len
falcon()                # call

🚀 Millennium Falcon | Power: 750 MW
Starship(name='Millennium Falcon', power=750)
🚀 Millennium Falcon+Viper Mk VIII | Power: 1350 MW
🚀 Millennium Falcon-Viper Mk VIII | Power: 150 MW
🚀 Millennium Falcon*2 | Power: 1500 MW
False
True
1500
🛸 Millennium Falcon launching manually...


At the academy, you’ve been assigned a critical mission:
Protect the warp drive so no cadet accidentally sets it to an unsafe speed.

So you create a security officer named WarpDrive that sits between the user and the actual value.

This officer checks every attempt to change the warp speed and either allows it or says:

“Captain, I cannot allow that!” 

That’s what a descriptor does — it controls access to an attribute.

🛠 Descriptor Basics
A descriptor is a class that implements any of these methods:

__get__() – for reading
__set__() – for writing
__delete__() – for deleting

In [1]:
class WarpDrive:
    def __init__(self, max_speed=10):
        self.max_speed = max_speed
        self._speed = 0

    def __get__(self, instance, owner):
        print("📡 Reading warp speed")
        return self._speed

    def __set__(self, instance, value):
        if value > self.max_speed:
            raise ValueError(f"Warp speed cannot exceed {self.max_speed}!")
        print("🛰 Setting warp speed")
        self._speed = value

In [2]:
class Starship:
    warp = WarpDrive(max_speed=8)

    def __init__(self, name):
        self.name = name

In [3]:
falcon = Starship("Millennium Falcon")
falcon.warp = 6
print(falcon.warp)  # Allowed

falcon.warp = 9     # Raises ValueError!

🛰 Setting warp speed
📡 Reading warp speed
6


ValueError: Warp speed cannot exceed 8!

In [4]:
class LifeSupport:
    def __init__(self):
        self._oxygen = 100

    def __get__(self, instance, owner):
        print("🫁 Checking oxygen levels...")
        return self._oxygen

    def __set__(self, instance, value):
        if not 0 <= value <= 100:
            raise ValueError("Oxygen must be between 0% and 100%")
        print("🧪 Adjusting oxygen levels...")
        self._oxygen = value

In [10]:
class ExplorerShip:
    life_support = LifeSupport()

    def __init__(self,name):
        self.name = name




In [11]:
ship = ExplorerShip()
ship.life_support = 75

TypeError: ExplorerShip.__init__() missing 1 required positional argument: 'name'

🏗️ What About @property?
If descriptors are custom gatekeepers , then @property is the built-in bouncer who can do basic checks without needing a whole class.

It's simpler, and perfect for controlling access to instance attributes .

In [12]:
class CargoShip:
    def __init__(self, fuel=100):
        self._fuel = fuel

    @property
    def fuel(self):
        print("⛽ Checking fuel reserves...")
        return self._fuel

    @fuel.setter
    def fuel(self, value):
        if value < 0:
            raise ValueError("Fuel level cannot be negative!")
        print("🔄 Refueling...")
        self._fuel = value

In [13]:
freighter = CargoShip()
print(freighter.fuel)      # Gets fuel
freighter.fuel = 80        # Sets fuel
freighter.fuel = -10       # Throws error

⛽ Checking fuel reserves...
100
🔄 Refueling...


ValueError: Fuel level cannot be negative!

In [14]:
from abc import ABC, abstractmethod

# Abstract Base Class
class Creature(ABC):
    @abstractmethod
    def move(self):
        pass

# Descriptors
class Secret:
    def __init__(self):
        self._value = None

    def __get__(self, instance, owner):
        return f"** Secret: {self._value} **"

    def __set__(self, instance, value):
        self._value = value.capitalize()

# Metaclass to auto-register classes
class MetaCreature(type):
    def __new__(cls, name, bases, attrs):
        print(f"Creating class {name} with MetaCreature")
        return super().__new__(cls, name, bases, attrs)

# Flying class
class Flyer:
    def move(self):
        return "Flying in the sky!"

# Swimmer class
class Swimmer:
    def move(self):
        return "Swimming in the water!"

# Hero inherits from both - Multiple Inheritance
class Hero(Flyer, Swimmer, metaclass=MetaCreature):
    secret = Secret()  # Using descriptor

    def __init__(self, name):
        self._name = name
        self.secret = "python wizard"

    def __str__(self):
        return f"Hero {self._name}"

    def __add__(self, other):
        return Hero(self._name + "-" + other._name)

# A class that must implement move()
class Bird(Creature):
    def move(self):
        return "Bird flaps wings and flies."

# --- Let's use the project ---

# Create Hero
hero1 = Hero("Zahid")
hero2 = Hero("Luna")

# Print hero
print(hero1)  # Uses __str__

# Operator overloading
hero3 = hero1 + hero2
print(hero3)

# MRO
print("Hero's move:", hero3.move())  # MRO chooses Flyer before Swimmer

# Abstract Base Class
bird = Bird()
print("Bird's move:", bird.move())

# Descriptor
print(hero3.secret)


Creating class Hero with MetaCreature
Hero Zahid
Hero Zahid-Luna
Hero's move: Flying in the sky!
Bird's move: Bird flaps wings and flies.
** Secret: Python wizard **
